# comfirm interpolation
### 2D in taichi

In [49]:
import taichi as ti
import taichi_glsl as ts
ti.init(arch = ti.cpu)

shape = (5, 5, 6)
field =  ti.field(ti.f64, shape = shape)
np.random.seed(3)
self = np.random.rand(shape[0], shape[1], shape[2])
field.from_numpy(self)
P = ti.Vector([1,1.5,2])



@ti.func
def trilerp(field: ti.template(), P):
    '''
    Tilinear sampling an 3D field with a real index.
    :parameter field: (3D Tensor)
        Specify the field to sample.
    :parameter P: (3D Vector of float)
        Specify the index in field.
    :note:
        If one of the element to be accessed is out of `field.shape`, then
        `Tilerp` will automatically do a clamp for you, see :func:`sample`.
        Syntax ref : https://en.wikipedia.org/wiki/Trilinear_interpolation.
    :return:
        The return value is calcuated as::
            I = int(P)
            w0 = ts.fract(P)
            w1 = 1.0 - w0
            c00 = ts.sample(field, I + ts.D.yyy) * w1.x + ts.sample(field, I + ts.D.xyy) * w0.x
            c01 = ts.sample(field, I + ts.D.yyx) * w1.x + ts.sample(field, I + ts.D.xyx) * w0.x
            c10 = ts.sample(field, I + ts.D.yxy) * w1.x + ts.sample(field, I + ts.D.xxy) * w0.x
            c11 = ts.sample(field, I + ts.D.yxx) * w1.x + ts.sample(field, I + ts.D.xxx) * w0.x
            c0 = c00 * w1.y + c10 * w0.y
            c1 = c01 * w1.y + c11 * w0.y
            return c0 * w1.z + c1 * w0.z
        .. where D = vec(1, 0, -1)
    '''
    I = int(P)
    w0 = ts.fract(P)
    w1 = 1.0 - w0

    c00 = ts.sample(field, I + ts.D.yyy) * w1.x + ts.sample(
        field, I + ts.D.xyy) * w0.x
    c01 = ts.sample(field, I + ts.D.yyx) * w1.x + ts.sample(
        field, I + ts.D.xyx) * w0.x
    c10 = ts.sample(field, I + ts.D.yxy) * w1.x + ts.sample(
        field, I + ts.D.xxy) * w0.x
    c11 = ts.sample(field, I + ts.D.yxx) * w1.x + ts.sample(
        field, I + ts.D.xxx) * w0.x

    c0 = c00 * w1.y + c10 * w0.y
    c1 = c01 * w1.y + c11 * w0.y

    return c0 * w1.z + c1 * w0.z


@ti.kernel
def main() -> ti.f64:
    # x =  ts.sampling.sample(field, P)
    x = trilerp(field, P)

    return x
# val = main()
# print(val)
main()

[Taichi] Starting on arch=x64


0.40962558119277803

### in numpy

In [50]:
import numpy as np
from scipy.interpolate import interpn

def field_point(self, coord): # It is normalized. To get original field: u.field_point()*u.magitude_point()
    points = (np.linspace(0,self.shape[0]-1,self.shape[0]), 
              np.linspace(0,self.shape[1]-1,self.shape[1]),
              np.linspace(0,self.shape[2]-1,self.shape[2])
            )
    fx = interpn(points, self, coord)[0]
    # fy = interpn(points, self.field_y, coord)[0]
    # fz = interpn(points, self.field_z, coord)[0]
    # return np.array([fx, fy, fz])
    return fx

shape = (5, 5, 6)
np.random.seed(3)
self = np.random.rand(shape[0], shape[1], shape[2])
print(field_point(self, (1, 1.5,2)))
# print(self[1,1,1])

0.40962558119277803


## 3D vector comfirmation

In [9]:
import taichi as ti
import taichi_glsl as ts
import numpy as np
ti.init(arch = ti.cpu)

shape = (5, 5, 6)
field =  ti.Vector.field(n = 3, dtype = ti.f64, shape = shape)
np.random.seed(3)
self = np.random.rand(shape[0], shape[1], shape[2])
temp = np.stack((self, self, self), axis = -1)
print(temp.shape)
field.from_numpy(temp)
P = ti.Vector([1,1.5,2])



@ti.func
def trilerp(field: ti.template(), P):
    I = int(P)
    w0 = ts.fract(P)
    w1 = 1.0 - w0

    c00 = ts.sample(field, I + ts.D.yyy) * w1.x + ts.sample(
        field, I + ts.D.xyy) * w0.x
    c01 = ts.sample(field, I + ts.D.yyx) * w1.x + ts.sample(
        field, I + ts.D.xyx) * w0.x
    c10 = ts.sample(field, I + ts.D.yxy) * w1.x + ts.sample(
        field, I + ts.D.xxy) * w0.x
    c11 = ts.sample(field, I + ts.D.yxx) * w1.x + ts.sample(
        field, I + ts.D.xxx) * w0.x

    c0 = c00 * w1.y + c10 * w0.y
    c1 = c01 * w1.y + c11 * w0.y

    return c0 * w1.z + c1 * w0.z


@ti.kernel
def main() -> ti.f64:
    # x =  ts.sampling.sample(field, P)
    x = trilerp(field, P)
    # return x # is not supported in taichi currently
    return x[0]
val = main()
print(val)
# main()

[Taichi] Starting on arch=x64
(5, 5, 6, 3)


ValueError: Invalid constant scalar expression: <class 'tuple'>